# [4.3 Solving a CSP using Search](http://artint.info/2e/html/ArtInt2e.Ch4.S3.html)
- [Implementation Details](http://artint.info/AIPython/aipython.pdf#page=56)

## About
We can already solve CSPs by using the search methods we learnt before. The search space is partial assignments to the variables in the CSP.

For example, imagine a CSP with two variables, $A$ and $B$, and domains $[1,2,3]$ and $[1]$ respectively. We start with the root node, `{}`, and fill in its children at the first level: `{A:1}`, `{A:2}`, and `{A:3}`. Then each of those nodes have another node, merging its own values with the values of the next variable as children: `{A:1, B:1}`, `{A:2, B:1}`, and `{A:3, B:1}`.

Because we are interested _if_ there is a solution, rather than the path to the solution (all solutions are the same length), and because the search space is acyclic, we can use depth-first search. As an optimization, before generating the neighboring nodes, we check if those nodes satisfy the constraints; if not, there is no point in going on further, as it will never lead to a solution.

## Instructions

Each section header contains a link to the corresponding chapter in the accompanying textbook, and an "Implementation Details" link provided throughout tells you how the implementation works. Before using this notebook, make sure you have followed the [installation instructions](https://aispace2.github.io/AISpace2/install.html) beforehand.

You can run each cell by selecting it and pressing *Ctrl+Enter*. Alternatively, you can click the *Play* button in the toolbar, to the left of the stop button. 

For more information, including how the code in this notebook differs from that in [AIPython](aipython.org), check out the [Reference](https://aispace2.github.io/AISpace2/reference.html).

## Searcher
Here is the searching algorithm we will use when solving CSPs with search. Depth-first search is a good choice because we know all solutions will be deep, and that there will be no cycles.

In [ ]:
from aipython.searchProblem import Path, Arc
from aipython.searchGeneric import Frontier
from aispace2.jupyter.search import Displayable, visualize

class Searcher(Displayable):
    """returns a searcher for a problem.
    Paths can be found by repeatedly calling search().
    This does depth-first search unless overridden
    """
    def __init__(self, problem):
        """creates a searcher from a problem
        """
        self.problem = problem
        self.initialize_frontier()
        self.num_expanded = 0
        self.add_to_frontier(Path(problem.start_node()))
        super().__init__()

    def initialize_frontier(self):
        self.frontier = []
        
    def empty_frontier(self):
        return self.frontier == []
        
    def add_to_frontier(self,path):
        self.frontier.append(path)
        
    @visualize
    def search(self):
        """returns (next) path from the problem's start node
        to a goal node. 
        Returns None if no path exists.
        """
        while not self.empty_frontier():
            path = self.frontier.pop()
            self.display(2, "Expanding:",path,"(cost:",path.cost,")")
            self.num_expanded += 1
            if self.problem.is_goal(path.end()):    # solution found
                self.display(1, self.num_expanded, "paths have been expanded and",
                            len(self.frontier), "paths remain in the frontier")
                self.solution = path   # store the solution found
            else:
                neighs = self.problem.neighbors(path.end())
                self.display(3,"Neighbors are", neighs)
                for arc in reversed(neighs):
                    self.add_to_frontier(Path(path,arc))
                self.display(3,"Frontier:",self.frontier)
        self.display(1,"No (more) solutions. Total of",
                     self.num_expanded,"paths expanded.")
        
class DF_branch_and_bound(Searcher):
    """returns a branch and bound searcher for a problem.    
    An optimal path with cost less than bound can be found by calling search()
    """
    def __init__(self, problem, bound=float("inf")):
        """creates a searcher than can be used with search() to find an optimal path.
        bound gives the initial bound. By default this is infinite - meaning there
        is no initial pruning due to depth bound
        """
        super().__init__(problem)
        self.best_path = None
        self.bound = bound

    @visualize
    def search(self):
        """returns an optimal solution to a problem with cost less than bound.
        returns None if there is no solution with cost less than bound."""
        self.frontier = [Path(self.problem.start_node())]
        self.num_expanded = 0
        while self.frontier:
            path = self.frontier.pop()
            if path.cost+self.problem.heuristic(path.end()) < self.bound:
                self.display(3,"Expanding:",path,"cost:",path.cost)
                self.num_expanded += 1
                if self.problem.is_goal(path.end()):
                    self.best_path = path
                    self.bound = path.cost
                    self.display(2,"New best path:",path," cost:",path.cost)
                    return
                neighs = self.problem.neighbors(path.end())
                self.display(3,"Neighbors are", neighs)
                for arc in reversed(list(neighs)):
                    self.add_to_frontier(Path(path, arc))
        self.display(1,"Number of paths expanded:",self.num_expanded)
        self.solution = self.best_path
        return self.best_path

## Converting the CSP to a Search problem

In order to use search algorithms on our CSP, we must first convert it into a `Search_problem`. We can do this by overriding the `neighbours` and `is_goal` methods. Notice how the neighbouring nodes are discovered on-the-fly.

In [ ]:
from aipython.searchProblem import Search_problem
from aipython.utilities import dict_union

class Search_from_CSP(Search_problem):
    """A search problem directly from the CSP.

    A node is a variable:value dictionary"""
    def __init__(self, csp, variable_order=None):
        self.csp=csp
        if variable_order:
            assert set(variable_order) == set(csp.variables)
            assert len(variable_order) == len(csp.variables)
            self.variables = variable_order
        else:
            self.variables = list(csp.variables)

    def is_goal(self, node):
        return len(node)==len(self.csp.variables)
    
    def start_node(self):
        return {}
    
    def neighbors(self, node):
        """iterator over the neighboring nodes of node"""
        var = self.variables[len(node)] # the next variable
        res = []
        for val in self.csp.domains[var]:
            new_env = dict_union(node,{var:val})  #dictionary union
            if self.csp.consistent(new_env):
                res.append(Arc(node,new_env))
        return res

In [ ]:
from aipython.cspExamples import simple_csp1, simple_csp2, extended_csp, crossword1, crossword2, crossword2d

search_csp = DF_branch_and_bound(Search_from_CSP(crossword1))
search_csp.show_edge_costs = False
search_csp.sleep_time = 0.5
# We know the search space is a tree.
search_csp.layout_method = "tree"
search_csp.search()
search_csp

In [ ]:
search_csp.search()